## Timing Comparison for Training Loops

In [1]:
import wandb
import config
import diamond_world_model_trainer as trainer
import incremental_training as incremental_trainer
import os
import shutil
import time
import gc
import torch

run = wandb.init(project='timing-comparison', reinit=True)

Imports successful.
Visualization helpers defined.
Training and validation epoch functions adapted for Batch object and Denoiser.forward.


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: irvin-hwang (irvin-hwang-simulacra-systems) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


### Run `_main_training` on non-incremental dataset

In [ ]:
gc.collect()
torch.cuda.empty_cache()

config.OUTPUT_DIR = os.path.join(config.AUXILIARY_DIR, 'output_model_2hz_DIAMOND_laundry_nonincremental_test')
config.DATA_DIR = os.path.join(config.AUXILIARY_DIR, 'jetbot_data_two_actions_nonincremental_test')
config.IMAGE_DIR = os.path.join(config.DATA_DIR, 'images')
config.CSV_PATH = os.path.join(config.DATA_DIR, 'laundry_data_incremental_test.csv')
config.NUM_EPOCHS = 1

if os.path.exists(config.OUTPUT_DIR):
    shutil.rmtree(config.OUTPUT_DIR)
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

start = time.time()
trainer_run = trainer._main_training(finish_run=False)
noninc_duration = time.time() - start
wandb.finish()

--- Main Training Execution --- 
--- Configuration ---
Using device: cuda
Configuration loaded for _main_training.
--- Initializing Models for _main_training ---
Denoiser model created for _main_training. Total parameter count: 330,464,131
DiffusionSampler created for visualization in _main_training.
--- Setting up Optimizer and Scheduler for _main_training ---
Optimizer: AdamW with LR=0.0001
LR Scheduler: LambdaLR with 100 warmup steps.
Wandb watching denoiser model.
No checkpoint found or specified for _main_training. Starting fresh.
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Full dataset size: 591
Creating new train/val split...
Saved new dataset split to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\output_model_2hz_DIAMOND_laundry_nonincremental_test\dataset_split.pth
Training dataset size: 531, Validation dataset size: 60
Preparing filtered validation subsets for visualization...
Filtering dataset with 60 samples for actions: 

Filtering Dataset:   0%|          | 0/60 [00:00<?, ?it/s]

Filtered down to 26 samples.
Filtering dataset with 60 samples for actions: [0.13]


Filtering Dataset:   0%|          | 0/60 [00:00<?, ?it/s]

Filtered down to 34 samples.
Found 26 stopped and 34 moving samples.
--- Starting Training Process in _main_training ---


Epoch 1 [Train]:   0%|          | 0/265 [00:00<?, ?it/s]

[Nonincremental Pre training loop] GPU mem alloc: 1263.9 MB, reserved: 1286.0 MB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1263 MiB |   1263 MiB |   1263 MiB |   2560 B   |
|       from large pool |   1248 MiB |   1248 MiB |   1248 MiB |      0 B   |
|       from small pool |     15 MiB |     15 MiB |     15 MiB |   2560 B   |
|---------------------------------------------------------------------------|
| Active memory         |   1263 MiB |   1263 MiB |   1263 Mi

### Run `train_diamond_model` on incremental dataset

In [3]:
gc.collect()
torch.cuda.empty_cache()

config.OUTPUT_DIR = os.path.join(config.AUXILIARY_DIR, 'output_model_2hz_DIAMOND_laundry_incremental_test')
config.DATA_DIR = os.path.join(config.AUXILIARY_DIR, 'jetbot_data_two_actions_incremental_test')
config.IMAGE_DIR = os.path.join(config.DATA_DIR, 'images')
config.CSV_PATH = os.path.join(config.DATA_DIR, 'laundry_data_incremental_test.csv')
config.EARLY_STOPPING_PATIENCE = 1

if os.path.exists(config.OUTPUT_DIR):
    shutil.rmtree(config.OUTPUT_DIR)
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

wandb.init(project='timing-comparison', reinit=True)
start = time.time()
incremental_trainer.main()
inc_duration = time.time() - start
wandb.finish()

Found 1 new sessions to collect.


Wrote 4538 entries to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_new_data_incremental_test\new.csv
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Fresh dataset size is 299.
Loaded combined CSV with columns: ['session_id', 'image_path', 'timestamp', 'action']
Full dataset size: 292
Creating new train/val split...
Saved new dataset split to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\output_model_2hz_DIAMOND_laundry_incremental_test\dataset_split.pth
Starting training
Using device: cuda
Loaded from checkpoint C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\output_model_2hz_DIAMOND_laundry_nonincremental_test\checkpoints\denoiser_model_best_val_loss.pth
Incremental training for at least 320 and at most 320 with 320 number of steps per fresh data epoch


Incremental Training Steps:   0%|          | 0/320 [00:00<?, ?it/s]

Filtering dataset with 30 samples for actions: [0.0]


Filtering Dataset:   0%|          | 0/30 [00:00<?, ?it/s]

...

Processing New Sessions:   0%|          | 0/1 [00:00<?, ?it/s]

Processing session: session_20250528_094602, 4538 entries.


  Processing session_20250528_094602:   0%|          | 0/4538 [00:00<?, ?it/s]

Appending 4538 new entries to C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\jetbot_data_two_actions_incremental_test\laundry_data_incremental_test.csv

Aggregate data saved. Total entries now: 8977
Session data combined.
Deleted dataset split file: C:\Projects\jetbot-diffusion-world-model-kong-finder-aux\output_model_2hz_DIAMOND_laundry_incremental_test\dataset_split.pth


A/avg_mse,█▅▁▆▂▂▃▃▃▂▂▄▄▄▄▅▄▄▄▄▅▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
A/avg_ssim,▁▃█▇▄▄▅▅▅▅▄▄▄▄▅▅▅▄▄▃▃▃▃▃▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▄
A/mse,▇▂▁▁▅▃▃▅▄▂▄▄▇▃▇▇▅▃▅▁▃▆▃▆▆▅▄▃▃█▅▆▅█▂▃▆▆▅█
A/ssim,▄▃▅▂▄▅▄▆▃▁▃▇▅▁▅▆▃▅▂▃▁▃▅█▄▃▅▆▃▃▂▃▃▇▆▄▃▆▅▃
B/avg_mse,█▁▂▂▂▂▂▁▁▂▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▄▄▅▅▅▅▅
B/avg_ssim,█▂▁▁▂▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▂▂▂▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁
B/mse,▄▇▂█▁▆▃▃▃▆▄▆▇▆▅▄█▇▆▆▅▅▅▆▄▅▅▅▃▁▄▆▃▇▂██▄▅▄
B/ssim,▃▂▇▂▃▂▂▅▃▄▃▁▁██▄▃▃▁▄▃▅▂▄▂▂▄▄▂▃▆▄▃▄▂▄▆▄▅▇
incremental_eval_val_loss,▁
incremental_fw_bw_duration,▇▇▇█▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
incremental_opt_sched_duration,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█


### Compare timings

In [4]:
import pandas as pd

comparison_df = pd.DataFrame({
    'run': ['non_incremental', 'incremental'],
    'duration_sec': [noninc_duration, inc_duration]
})
comparison_df

,run,duration_sec
0,non_incremental,116.087261
1,incremental,3674.671787
